In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from underthesea import word_tokenize, pos_tag, sent_tokenize
from pyvi.ViTokenizer import tokenize
from pyvi import ViTokenizer
from underthesea import word_tokenize
import warnings
from gensim import corpora, models, similarities
from tqdm import tqdm

import re

warnings.filterwarnings("ignore")

In [62]:
df = pd.read_excel('data_motobikes.xlsx')

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7208 entries, 0 to 7207
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   7208 non-null   int64 
 1   Tiêu đề              7207 non-null   object
 2   Giá                  7206 non-null   object
 3   Khoảng giá min       7006 non-null   object
 4   Khoảng giá max       7011 non-null   object
 5   Địa chỉ              7167 non-null   object
 6   Mô tả chi tiết       7208 non-null   object
 7   Thương hiệu          7208 non-null   object
 8   Dòng xe              7208 non-null   object
 9   Năm đăng ký          7208 non-null   object
 10  Số Km đã đi          7208 non-null   int64 
 11  Tình trạng           7208 non-null   object
 12  Loại xe              7208 non-null   object
 13  Dung tích xe         7208 non-null   object
 14  Xuất xứ              7208 non-null   object
 15  Chính sách bảo hành  7207 non-null   object
 16  Trọng 

In [64]:
df.columns

Index(['id', 'Tiêu đề', 'Giá', 'Khoảng giá min', 'Khoảng giá max', 'Địa chỉ',
       'Mô tả chi tiết', 'Thương hiệu', 'Dòng xe', 'Năm đăng ký',
       'Số Km đã đi', 'Tình trạng', 'Loại xe', 'Dung tích xe', 'Xuất xứ',
       'Chính sách bảo hành', 'Trọng lượng', 'Href'],
      dtype='object')

In [65]:
data = df[['id', 'Tiêu đề','Mô tả chi tiết']]
data.head()

,id,Tiêu đề,Mô tả chi tiết
0,1,Ban xe,"Do mình dư xe không dùng nữa, xe máy móc vẫn c..."
1,2,Cần bán chiếc aprillia gt rs200 xe trùm mềm,Xe nhà trùm mềm chỉ đẩy ra đẩy vô nay phụ huyn...
2,3,Bán xe Sr Gt 200 ít sử dụng do đi oto,"Xe biển hcm, ít sử dụng còn rất mới\nLiên hệ: ***"
3,4,Nhà dư cần bán,Nhà dư chiếc xe wave có cavet. Máy móc êm. Có ...
4,5,cần bán,Mình kẹt tiền cần bán ai quan tâm inbox mình x...


In [66]:
data['Content'] = data['Mô tả chi tiết'].apply(lambda x: ' '.join(x.split()[:200]))
data.head()

,id,Tiêu đề,Mô tả chi tiết,Content
0,1,Ban xe,"Do mình dư xe không dùng nữa, xe máy móc vẫn c...","Do mình dư xe không dùng nữa, xe máy móc vẫn c..."
1,2,Cần bán chiếc aprillia gt rs200 xe trùm mềm,Xe nhà trùm mềm chỉ đẩy ra đẩy vô nay phụ huyn...,Xe nhà trùm mềm chỉ đẩy ra đẩy vô nay phụ huyn...
2,3,Bán xe Sr Gt 200 ít sử dụng do đi oto,"Xe biển hcm, ít sử dụng còn rất mới\nLiên hệ: ***","Xe biển hcm, ít sử dụng còn rất mới Liên hệ: ***"
3,4,Nhà dư cần bán,Nhà dư chiếc xe wave có cavet. Máy móc êm. Có ...,Nhà dư chiếc xe wave có cavet. Máy móc êm. Có ...
4,5,cần bán,Mình kẹt tiền cần bán ai quan tâm inbox mình x...,Mình kẹt tiền cần bán ai quan tâm inbox mình x...


In [67]:
data.columns

Index(['id', 'Tiêu đề', 'Mô tả chi tiết', 'Content'], dtype='object')

In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7208 entries, 0 to 7207
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              7208 non-null   int64 
 1   Tiêu đề         7207 non-null   object
 2   Mô tả chi tiết  7208 non-null   object
 3   Content         7208 non-null   object
dtypes: int64(1), object(3)
memory usage: 225.4+ KB


In [69]:
stop_word_file = 'files/vietnamese-stopwords.txt'
emojicon_file = 'files/emojicon.txt'
teencode_file = 'files/teencode.txt'


In [70]:
with open(stop_word_file, 'r', encoding='utf-8') as f:
    stopwords = set([w.strip() for w in f.readlines() if w.strip()])

with open(emojicon_file, 'r', encoding='utf-8') as f:
    emojicons = [w.strip() for w in f.readlines() if w.strip()]

with open(teencode_file, 'r', encoding='utf-8') as f:
    teencode_map = {}
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 2:
            teencode_map[parts[0]] = " ".join(parts[1:])


special_tokens = ['', ' ', ',', '.', '...', '-', ':', ';', '?', '%', '(', ')', '+', '/', "'", '&', '#', '*', '!', '"', '_', '=', '[', ']', '{', '}', '~', '`', '|', '\\']


print(f"Stopwords: {len(stopwords)}, Emojis: {len(emojicons)}, Teencode: {len(teencode_map)}")


Stopwords: 1957, Emojis: 67, Teencode: 416


In [71]:
# Các hàm xử lý
def remove_emojis(text):
    for emo in emojicons:
        text = text.replace(emo, ' ')
    return text

def normalize_teencode(text):
    for key, val in teencode_map.items():
        text = re.sub(rf'\b{re.escape(key)}\b', val, text)
    return text

def remove_special_chars(text):
    text = re.sub(r'[^\w\s]', ' ', text)  # loại ký tự đặc biệt
    text = re.sub(r'\s+', ' ', text).strip()  # loại khoảng trắng thừa
    return text

# -----------------------
# 4. TÁCH STOPWORD RIÊNG
# -----------------------
def remove_stopwords(text):
    tokens = word_tokenize(text, format="text").split()
    tokens = [t for t in tokens if t not in stopwords]
    return ' '.join(tokens)

# -----------------------
# 5. CHUẨN HÓA TỔNG HỢP
# -----------------------
def clean_text(text):
    text = str(text).lower()
    text = remove_emojis(text)
    text = normalize_teencode(text)
    text = remove_special_chars(text)
    text = remove_stopwords(text)
    return text

In [72]:
data['clean_text'] = data['Content'].apply(clean_text)
data['clean_text']

0       dư xe xe_máy_móc zin êm_ru thanh_lí gấp điện_t...
1       xe trùm mềm đẩy đẩy vô phụ_huynh lớn_tuổi đẩy ...
2                                     xe biển hcm liên_hệ
3       dư xe wave cavet máy_móc êm bình đèn đi_lại li...
4                                       kẹt tiền inbox xe
                              ...                        
7203                        xe_máy độ hàng chuẩn sử_dunhj
7204    lớn_tuổi bảo dưởng xe kỷ_vỏ ruột_mơi 97 xe chủ...
7205    ex 2017 bs93 de 9 chủ máy_móc zin nồi zin bao ...
7206    đời sh_ý đời 2010 máy 105 máy_móc nguyên_zin m...
7207    xe ngoại_hình đẹp liền lạc_máy êm xe mua_bán b...
Name: clean_text, Length: 7208, dtype: object

In [73]:
data.head()

,id,Tiêu đề,Mô tả chi tiết,Content,clean_text
0,1,Ban xe,"Do mình dư xe không dùng nữa, xe máy móc vẫn c...","Do mình dư xe không dùng nữa, xe máy móc vẫn c...",dư xe xe_máy_móc zin êm_ru thanh_lí gấp điện_t...
1,2,Cần bán chiếc aprillia gt rs200 xe trùm mềm,Xe nhà trùm mềm chỉ đẩy ra đẩy vô nay phụ huyn...,Xe nhà trùm mềm chỉ đẩy ra đẩy vô nay phụ huyn...,xe trùm mềm đẩy đẩy vô phụ_huynh lớn_tuổi đẩy ...
2,3,Bán xe Sr Gt 200 ít sử dụng do đi oto,"Xe biển hcm, ít sử dụng còn rất mới\nLiên hệ: ***","Xe biển hcm, ít sử dụng còn rất mới Liên hệ: ***",xe biển hcm liên_hệ
3,4,Nhà dư cần bán,Nhà dư chiếc xe wave có cavet. Máy móc êm. Có ...,Nhà dư chiếc xe wave có cavet. Máy móc êm. Có ...,dư xe wave cavet máy_móc êm bình đèn đi_lại li...
4,5,cần bán,Mình kẹt tiền cần bán ai quan tâm inbox mình x...,Mình kẹt tiền cần bán ai quan tâm inbox mình x...,kẹt tiền inbox xe


In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7208 entries, 0 to 7207
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              7208 non-null   int64 
 1   Tiêu đề         7207 non-null   object
 2   Mô tả chi tiết  7208 non-null   object
 3   Content         7208 non-null   object
 4   clean_text      7208 non-null   object
dtypes: int64(1), object(4)
memory usage: 281.7+ KB


In [75]:
data.columns

Index(['id', 'Tiêu đề', 'Mô tả chi tiết', 'Content', 'clean_text'], dtype='object')

In [76]:
vectorizer = TfidfVectorizer(
    analyzer='word',
    max_features=8000
)
tfidf_matrix = vectorizer.fit_transform(data['clean_text'])
print("Shape TF-IDF:", tfidf_matrix.shape)

Shape TF-IDF: (7208, 8000)


### Consin Simalarity

In [77]:
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
print("Cosine similarity matrix shape:", cosine_sim_matrix.shape)

Cosine similarity matrix shape: (7208, 7208)


In [78]:
data_show = pd.DataFrame(cosine_sim_matrix)
data_show

,0,1,2,3,4,5,6,7,8,9,...,7198,7199,7200,7201,7202,7203,7204,7205,7206,7207
0,1.000000,0.002356,0.014869,0.088510,0.010484,0.005733,0.17793,0.007029,0.023749,0.028089,...,0.000000,0.000000,0.004611,0.009947,0.057697,0.0,0.007941,0.032060,0.004306,0.010897
1,0.002356,1.000000,0.006119,0.002910,0.056636,0.002359,0.00000,0.002893,0.065462,0.024339,...,0.000000,0.000000,0.011288,0.024966,0.007490,0.0,0.050077,0.001363,0.049125,0.004484
2,0.014869,0.006119,1.000000,0.124109,0.027230,0.100600,0.00000,0.018257,0.061685,0.148400,...,0.000000,0.000000,0.011977,0.025834,0.325652,0.0,0.082709,0.008602,0.075559,0.028303
3,0.088510,0.002910,0.124109,1.000000,0.012952,0.047850,0.00000,0.008684,0.029340,0.041846,...,0.000000,0.020938,0.043815,0.012288,0.087206,0.0,0.009810,0.029263,0.068662,0.069161
4,0.010484,0.056636,0.027230,0.012952,1.000000,0.010498,0.00000,0.012872,0.043494,0.084242,...,0.000000,0.000000,0.008445,0.018216,0.033333,0.0,0.014542,0.006065,0.007885,0.019956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7203,0.000000,0.000000,0.000000,0.000000,0.000000,0.084680,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.038760,0.000000,1.0,0.000000,0.000000,0.000000,0.000000
7204,0.007941,0.050077,0.082709,0.009810,0.014542,0.039536,0.00000,0.009750,0.089794,0.029163,...,0.000000,0.000000,0.006396,0.013797,0.063248,0.0,1.000000,0.012522,0.016279,0.015116
7205,0.032060,0.001363,0.008602,0.029263,0.006065,0.003317,0.00000,0.004067,0.061164,0.035200,...,0.000000,0.009806,0.021835,0.059649,0.056232,0.0,0.012522,1.000000,0.026413,0.023780
7206,0.004306,0.049125,0.075559,0.068662,0.007885,0.029132,0.00000,0.005287,0.181294,0.080231,...,0.044423,0.012747,0.003468,0.048566,0.053093,0.0,0.016279,0.026413,1.000000,0.008196


In [79]:
def recommend(item_id: int, top_n: int = 5):
    """
    Recommend similar motorbikes based on cosine similarity.
    Args:
        item_id (int): id hoặc index của xe trong DataFrame
        top_n (int): số lượng gợi ý muốn lấy
    Returns:
        DataFrame chứa các xe tương tự
    """
    if item_id not in data.index:
        raise ValueError(f"id {item_id} không tồn tại trong DataFrame")

    # Lấy hàng tương ứng trong ma trận cosine
    sim_scores = list(enumerate(cosine_sim_matrix[item_id]))

    # Sắp xếp theo độ tương đồng giảm dần, bỏ chính nó
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1: top_n + 1]

    # Lấy index xe tương tự
    similar_indices = [i[0] for i in sim_scores]
    similar_scores = [i[1] for i in sim_scores]

    # Tạo DataFrame kết quả
    recommendations = data.loc[similar_indices, ['id', 'Tiêu đề', 'Content']].copy()
    recommendations['similarity'] = similar_scores
    return recommendations.reset_index(drop=True)

In [80]:
def recommend_cosine(item_id: int, top_n: int = 5):
    sim_scores = cosine_sim_matrix[item_id]
    sim_scores[item_id] = -1  # bỏ chính nó

    similar_idx = sim_scores.argsort()[::-1][:top_n]
    similar_scores = sim_scores[similar_idx]

    results = data.iloc[similar_idx][['id', 'Tiêu đề', 'Content']].copy()
    results['similarity'] = similar_scores
    return results.reset_index(drop=True)

In [81]:
sample_id = 3
print(f"\nXe gốc:\n{data.loc[sample_id, ['id', 'Tiêu đề', 'Content']]}")
print("\nGợi ý các xe tương tự:")
recommendation = recommend(sample_id, top_n=5)
recommendation


Xe gốc:
id                                                         4
Tiêu đề                                       Nhà dư cần bán
Content    Nhà dư chiếc xe wave có cavet. Máy móc êm. Có ...
Name: 3, dtype: object

Gợi ý các xe tương tự:


,id,Tiêu đề,Content,similarity
0,3797,cần bán xe wave s110,Nhà dư xe không đi cần bán lại xe wave s110 Ac...,0.441712
1,4644,BÁN XE WINNER X V2 2020 DƯ DÙNG,"Ngoại hình như hình, máy móc êm Nhà đang dư cầ...",0.332510
2,6062,xe khong di ban lai,Nhà dư xe bán lại cho ai cần đi chơi,0.330674
3,3645,Wave A Nhật còn chạy tốt BSTP,Mình cần để lại chiếc Wave A Nhật máy móc còn ...,0.328840
4,4346,Thanh Lý xe Wave RSX,Dư xe cần thanh lý xe Wave RSX chính chủ . Liê...,0.326843


In [82]:
import joblib

# 2️ Lưu cosine matrix và vectorizer để tái sử dụng
joblib.dump(cosine_sim_matrix, 'cosine_sim_matrix.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
print("Đã lưu cosine_sim_matrix.pkl và tfidf_vectorizer.pkl")


Đã lưu cosine_sim_matrix.pkl và tfidf_vectorizer.pkl


In [83]:
def load_cosine_data():
    """
    Load cosine similarity matrix và vectorizer đã lưu trước đó.
    """
    cosine_matrix = joblib.load('cosine_sim_matrix.pkl')
    vectorizer_loaded = joblib.load('tfidf_vectorizer.pkl')
    print("Đã load cosine matrix & vectorizer thành công.")
    return cosine_matrix, vectorizer_loaded

# Gensim

In [84]:
content_gem = [[text for text in x.split()] for x in data.clean_text]

In [85]:
len(content_gem)

7208

In [86]:
content_gem[:1]

[['dư',
  'xe',
  'xe_máy_móc',
  'zin',
  'êm_ru',
  'thanh_lí',
  'gấp',
  'điện_thoại',
  'phú']]

In [87]:
dictionary = corpora.Dictionary(content_gem)

In [88]:
dictionary.token2id

{'dư': 0,
 'gấp': 1,
 'phú': 2,
 'thanh_lí': 3,
 'xe': 4,
 'xe_máy_móc': 5,
 'zin': 6,
 'êm_ru': 7,
 'điện_thoại': 8,
 '5000': 9,
 '89': 10,
 'bớt': 11,
 'cam_đẹp': 12,
 'chút': 13,
 'giá': 14,
 'kiếm': 15,
 'km': 16,
 'lớn_tuổi': 17,
 'mong': 18,
 'màu': 19,
 'mún': 20,
 'mềm': 21,
 'nỗi': 22,
 'odo': 23,
 'phụ_huynh': 24,
 'tiền': 25,
 'tr': 26,
 'trùm': 27,
 'vô': 28,
 'xăng': 29,
 'đẩy': 30,
 'đỏ': 31,
 'biển': 32,
 'hcm': 33,
 'liên_hệ': 34,
 'bình': 35,
 'cavet': 36,
 'máy_móc': 37,
 'wave': 38,
 'êm': 39,
 'đi_lại': 40,
 'đèn': 41,
 'inbox': 42,
 'kẹt': 43,
 '50': 44,
 'bs': 45,
 'cc': 46,
 'chí_minh': 47,
 'chức_năng': 48,
 'hình': 49,
 'hồ': 50,
 'keng': 51,
 'leng': 52,
 'sang_tên': 53,
 'thành_phố': 54,
 'xe_máy': 55,
 'đầy_đủ': 56,
 'anh_chị_em': 57,
 'thích': 58,
 'con_nhỏ': 59,
 'kho': 60,
 'thích_hợp': 61,
 'đổi': 62,
 '3': 63,
 '9': 64,
 'chủ': 65,
 'x': 66,
 '000': 67,
 '2023': 68,
 '52': 69,
 '59': 70,
 '7': 71,
 '90': 72,
 'a3': 73,
 'benelli': 74,
 'chi_tiết': 75,
 

In [89]:
feature_cnt = len(dictionary.token2id)
feature_cnt

12231

In [90]:
corpus = [dictionary.doc2bow(text) for text in content_gem]

In [91]:
corpus[1]

[(4, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 2),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 3),
 (31, 1)]

In [92]:
# Use TF-IDF Model to process corpus, obtaining index
tfidf = models.TfidfModel(corpus)
# tính toán sự tương tự trong ma trận thưa thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus],
                                            num_features = feature_cnt)
# ma tran: n x n

In [93]:
data_1 = pd.DataFrame(index)
data_1

,0,1,2,3,4,5,6,7,8,9,...,7198,7199,7200,7201,7202,7203,7204,7205,7206,7207
0,1.000000,0.000046,0.000363,0.070062,0.000226,0.000128,0.165256,0.000140,0.000168,0.007522,...,0.000000,0.000000,0.000077,0.000192,0.020802,0.0,0.000137,0.011216,0.000078,0.000233
1,0.000046,1.000000,0.000147,0.000064,0.043670,0.000052,0.000000,0.000057,0.010933,0.013176,...,0.000000,0.000000,0.005660,0.014739,0.000162,0.0,0.039439,0.000022,0.029932,0.000094
2,0.000363,0.000147,1.000000,0.081555,0.000717,0.066468,0.000000,0.000444,0.000534,0.099120,...,0.000000,0.000000,0.000244,0.000610,0.215618,0.0,0.038390,0.000174,0.040551,0.000740
3,0.070062,0.000064,0.081555,1.000000,0.000310,0.028778,0.000000,0.000192,0.000231,0.017177,...,0.000000,0.011990,0.028901,0.000264,0.045319,0.0,0.000189,0.013663,0.036919,0.045308
4,0.000226,0.043670,0.000717,0.000310,1.000000,0.000253,0.000000,0.000276,0.000332,0.059526,...,0.000000,0.000000,0.000152,0.000380,0.000792,0.0,0.000271,0.000108,0.000154,0.000460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7203,0.000000,0.000000,0.000000,0.000000,0.000000,0.073124,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.029468,0.000000,1.0,0.000000,0.000000,0.000000,0.000000
7204,0.000137,0.039439,0.038390,0.000189,0.000271,0.020485,0.000000,0.000168,0.006982,0.005477,...,0.000000,0.000000,0.000092,0.000231,0.021444,0.0,1.000000,0.002278,0.003245,0.000280
7205,0.011216,0.000022,0.000174,0.013663,0.000108,0.000061,0.000000,0.000067,0.043382,0.011344,...,0.000000,0.004184,0.011069,0.038818,0.019781,0.0,0.002278,1.000000,0.009314,0.008668
7206,0.000078,0.029932,0.040551,0.036919,0.000154,0.014309,0.000000,0.000096,0.026207,0.046227,...,0.012404,0.005962,0.000053,0.027593,0.022534,0.0,0.003245,0.009314,1.000000,0.000159


In [94]:
# giả sử df_1 là ma trận tương đồng n×n, index và columns đều là chỉ số/ID
id = 3
row = data_1.loc[id]

top5 = row.drop(id, errors='ignore').nlargest(5)   # bỏ chính nó
top5_indices = top5.index.tolist()                # <-- lấy index của Series
print(top5_indices)   
data.iloc[top5_indices]                            

[3796, 7115, 6061, 907, 3510]


,id,Tiêu đề,Mô tả chi tiết,Content,clean_text
3796,3797,cần bán xe wave s110,Nhà dư xe không đi cần bán lại xe wave s110 \n...,Nhà dư xe không đi cần bán lại xe wave s110 Ac...,dư xe xe wave s110 anh_chị_em xe đi_lại liên_h...
7115,7116,Xe Sirius RC Fi,Xe dùng để đi lại đổi xe nên muốn bán\nGiấy tờ...,Xe dùng để đi lại đổi xe nên muốn bán Giấy tờ ...,xe đi_lại đổi xe giấy_tờ chủ cũ đầy_đủ
6061,6062,xe khong di ban lai,Nhà dư xe bán lại cho ai cần đi chơi,Nhà dư xe bán lại cho ai cần đi chơi,dư xe
907,908,"Cần bán xe chinh chủ , biển số thành phố","xe đi còn êm , đi lại mượt. \nxe hay bảo trì h...","xe đi còn êm , đi lại mượt. xe hay bảo trì hàn...",xe êm đi_lại mượt xe bảo_trì hàng
3510,3511,do không có nhu cầu dùng nữa lên muống nhượng lại,Xe máy móc êm\nĐi lại bth \nAn ninh tốt,Xe máy móc êm Đi lại bth An ninh tốt,xe_máy_móc êm đi_lại bình_thường an_ninh tốt


In [95]:
# Trường hợp khách hàng nhập thông tin tìm kếm
search_str = "xe chính chủ, nguyên kiện"
search_str_wt = clean_text(search_str)
print(search_str_wt.split())
# content_gem_re[:1]

['xe', 'chủ_nguyên', 'kiện']


In [96]:
view_content = search_str_wt.split()
kw_vector = dictionary.doc2bow(view_content)
sim = index[tfidf[kw_vector]]

In [97]:
# sim là numpy array chứa độ tương đồng
# Tạo DataFrame gồm 2 cột: id và sim
df_sim = pd.DataFrame({
    "id": range(len(sim)),
    "sim": sim
})

# Sắp xếp theo sim giảm dần
df_sorted_search = df_sim.sort_values(by="sim", ascending=False)
recommend = df_sorted_search.head()
recommend

,id,sim
4467,4467,0.514529
3989,3989,0.428086
1961,1961,0.291052
6132,6132,0.273028
3789,3789,0.247979


In [98]:
data.iloc[recommend["id"]]

,id,Tiêu đề,Mô tả chi tiết,Content,clean_text
4467,4468,"Bán xe Winner 150 chính chủ,nguyên zin","Bán xe winner 150 chính chủ.nguyên zin, mới le...","Bán xe winner 150 chính chủ.nguyên zin, mới le...",xe winner 150 chủ_nguyên zin leng keng liên_hệ
3989,3990,Cần bán gấp wave chính chủ 110 2023 bstp,Wave 110 đời 2023\nodo 19k\nbiển số tp chính c...,Wave 110 đời 2023 odo 19k biển số tp chính chủ...,wave 110 đời 2023 odo 19 k biển tp chủ_nguyên ...
1961,1962,Future led 8/2022 bstp chính chủ xe ít sử dụng,Future led bstp chính chủ 8/2022 đầy đủ phụ ki...,Future led bstp chính chủ 8/2022 đầy đủ phụ ki...,future led bstp chủ 8 2022 đầy_đủ phụ kiện xe ...
6132,6133,Chính chủ cần bán Exciter 150 2017 máy zin,Bán xe exciter 150 chính chủ nguyên zin\n- Biể...,Bán xe exciter 150 chính chủ nguyên zin - Biển...,xe exciter 150 chủ_nguyên zin biển lâm_đồng 49...
3789,3790,"Cần bán Wave Rs bản 2010 , xe 1 đời chủ nguyên...",Cần bán Wave Rs 110 đời 2010 xe 1 đời chủ nguy...,Cần bán Wave Rs 110 đời 2010 xe 1 đời chủ nguy...,wave rs 110 đời 2010 xe 1 đời chủ_nguyên zin h...


In [99]:
class Recommender:
    def __init__(self, dictionary, tfidf, index, data: pd.DataFrame, clean_text):
        self.dictionary = dictionary
        self.tfidf = tfidf
        self.index = index
        self.data = data
        self.clean_text = clean_text

    def recommend_by_text(self, search_str: str, top_n: int = 5) -> pd.DataFrame:
        """Gợi ý theo nội dung văn bản."""
        search_str_wt = self.clean_text(search_str)
        tokens = search_str_wt.split()
        kw_vector = self.dictionary.doc2bow(tokens)
        sim = self.index[self.tfidf[kw_vector]]
        df_sim = pd.DataFrame({"id": range(len(sim)), "sim": sim}).sort_values(by="sim", ascending=False)
        recommend = df_sim.head(top_n)
        results = self.data.iloc[recommend["id"].to_list()].copy()
        results["sim"] = recommend["sim"].values
        return results

    def recommend_by_id(self, id: int, top_n: int = 5) -> pd.DataFrame:
        """Gợi ý các tài liệu tương tự với doc_id trên ma trận tương đồng n×n."""
        # Lấy dòng tương đồng từ ma trận
        data_1 = pd.DataFrame(self.index)
        try:
            row = data_1.loc[id]
        except KeyError:
            raise ValueError(f"id {id} không tồn tại trong ma trận tương đồng.")

        # Bỏ chính nó, lấy top N
        top_n_sim = row.drop(id, errors="ignore").nlargest(top_n)
        top_n_indices = top_n_sim.index.tolist()

        # Truy xuất dữ liệu tương ứng
        results = self.data.iloc[top_n_indices].copy()
        results["sim"] = top_n_sim.values

        return results

In [100]:
# giả sử df_1 là ma trận tương đồng n×n, index và columns đều là chỉ số/ID
id = 0
row = data_1.loc[id]

top5 = row.drop(id, errors='ignore').nlargest(5)   # bỏ chính nó
top5_indices = top5.index.tolist()                 # <-- lấy index của Series
print(top5_indices)                               # [chỉ số các item tương tự nhất]

[6992, 837, 445, 5639, 5657]


In [101]:
rec = Recommender(dictionary, tfidf, index, data, clean_text)
recommend_by_text = rec.recommend_by_text
recommend_by_id = rec.recommend_by_id

In [112]:
search_str = "xe còn mới, máy êm, hao xăng ít, đời từ 2019 trở lên. Nếu có Vision hoặc Janus chạy dưới 10.000km thì càng tốt."
recommend_by_text(search_str)


,id,Tiêu đề,Mô tả chi tiết,Content,clean_text,sim
6447,6448,Xe janus xám đen bảng smartkey,Cần bán xe janus \nDo mua xe mới nên cần bán lại,Cần bán xe janus Do mua xe mới nên cần bán lại,xe janus mua xe,0.447361
6442,6443,Cần bán xe tay ga Yamaha Janus,Xe chính chủ cần bán xe Yamaha Janus ai cần mu...,Xe chính chủ cần bán xe Yamaha Janus ai cần mu...,xe chủ xe yamaha janus mua liên_hệ xe,0.349798
6392,6393,janus,Janus đỏ đen đèn rất sáng giấy tờ đầy đủ,Janus đỏ đen đèn rất sáng giấy tờ đầy đủ,janus đỏ đen đèn giấy_tờ đầy_đủ,0.304264
6401,6402,Bán Janus xe nữ đi đầu đủ giấy tờ,Bán Janus xe nữ đi đầu đủ giấy tờ \nGiá cả thư...,Bán Janus xe nữ đi đầu đủ giấy tờ Giá cả thươn...,janus xe nữ đầu giấy_tờ giá_cả thương_lượng,0.284921
3613,3614,Bán Vision 215 màu đỏ biển tp thì chia lại cho e,Anh chị nào muốn lên đời hoặc nhà dư vision đờ...,Anh chị nào muốn lên đời hoặc nhà dư vision đờ...,anh_chị đời nhà_dư vision đời 215 trở màu đỏ b...,0.269901


In [103]:
id = 3
sample_id = df.loc[id, 'id']
print(f"\nXe gốc:\n{df.loc[df['id'] == sample_id, ['id', 'Tiêu đề', 'Mô tả chi tiết']].iloc[0]}")
print("\nGợi ý các xe tương tự:")
recommend_by_id(id)


Xe gốc:
id                                                                4
Tiêu đề                                              Nhà dư cần bán
Mô tả chi tiết    Nhà dư chiếc xe wave có cavet. Máy móc êm. Có ...
Name: 3, dtype: object

Gợi ý các xe tương tự:


,id,Tiêu đề,Mô tả chi tiết,Content,clean_text,sim
3796,3797,cần bán xe wave s110,Nhà dư xe không đi cần bán lại xe wave s110 \n...,Nhà dư xe không đi cần bán lại xe wave s110 Ac...,dư xe xe wave s110 anh_chị_em xe đi_lại liên_h...,0.418965
7115,7116,Xe Sirius RC Fi,Xe dùng để đi lại đổi xe nên muốn bán\nGiấy tờ...,Xe dùng để đi lại đổi xe nên muốn bán Giấy tờ ...,xe đi_lại đổi xe giấy_tờ chủ cũ đầy_đủ,0.334795
6061,6062,xe khong di ban lai,Nhà dư xe bán lại cho ai cần đi chơi,Nhà dư xe bán lại cho ai cần đi chơi,dư xe,0.310464
907,908,"Cần bán xe chinh chủ , biển số thành phố","xe đi còn êm , đi lại mượt. \nxe hay bảo trì h...","xe đi còn êm , đi lại mượt. xe hay bảo trì hàn...",xe êm đi_lại mượt xe bảo_trì hàng,0.309105
3510,3511,do không có nhu cầu dùng nữa lên muống nhượng lại,Xe máy móc êm\nĐi lại bth \nAn ninh tốt,Xe máy móc êm Đi lại bth An ninh tốt,xe_máy_móc êm đi_lại bình_thường an_ninh tốt,0.308166


In [104]:
def evaluate_cosine(data, cosine_sim_matrix, sample_size=200):
    indices = np.random.choice(data.index, sample_size, replace=False)

    top1_scores = []
    top3_scores = []
    times = []

    for idx in tqdm(indices, desc="Evaluating Cosine"):
        t0 = time.time()

        sim_scores = cosine_sim_matrix[idx]
        sim_scores[idx] = -1  # loại chính nó

        top3_idx = np.argsort(sim_scores)[-3:][::-1]
        top3_sim = sim_scores[top3_idx]

        t1 = time.time() - t0
        times.append(t1)

        top1_scores.append(top3_sim[0])
        top3_scores.append(np.mean(top3_sim))

    return {
        "Model": "Cosine-Similarity",
        "Avg_Time": np.mean(times),
        "Avg_Top1_Sim": np.mean(top1_scores),
        "Avg_Top3_Sim": np.mean(top3_scores)
    }



In [105]:
def evaluate_gensim(model, data, sample_size=200):
    indices = np.random.choice(len(data), size=sample_size, replace=False)

    top1_scores = []
    top3_scores = []
    times = []

    for idx in tqdm(indices, desc="Evaluating Gensim"):
        t0 = time.time()
        recs = model.recommend_by_id(idx, top_n=3)  # dùng đúng hàm của bạn
        t1 = time.time() - t0
        times.append(t1)

        top1_scores.append(recs.iloc[0]["sim"])
        top3_scores.append(recs["sim"].mean())

    return {
        "Model": "Gensim",
        "Avg_Time": np.mean(times),
        "Avg_Top1_Sim": np.mean(top1_scores),
        "Avg_Top3_Sim": np.mean(top3_scores)
    }


In [106]:
import random
import time

results = []

res_gensim = evaluate_gensim(rec, data, sample_size=10)
results.append(res_gensim)

res_cosine = evaluate_cosine(data, cosine_sim_matrix, sample_size=10)
results.append(res_cosine)

df_results = pd.DataFrame(results)
df_results



Evaluating Cosine: 100%|██████████| 10/10 [00:00<00:00, 2486.84it/s]


,Model,Avg_Time,Avg_Top1_Sim,Avg_Top3_Sim
0,Gensim,8.697196,0.350738,0.312288
1,Cosine-Similarity,0.000402,0.396365,0.344270


In [108]:
def recommend_cosine_by_text(query: str, top_n: int = 5):
    """
    Gợi ý xe máy tương tự dựa trên văn bản người dùng nhập vào.
    
    Args:
        query (str): văn bản tìm kiếm
        top_n (int): số lượng gợi ý
    
    Returns:
        DataFrame: danh sách xe tương tự + độ tương đồng
    """

    # 1. Tiền xử lý query bằng hàm clean_text của bạn
    clean_query = clean_text(query)

    # 2. Vector hóa query
    query_vec = vectorizer.transform([clean_query])

    # 3. Tính độ tương đồng cosine giữa query và toàn bộ item
    sims = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # 4. Lấy top N kết quả cao nhất
    top_idx = sims.argsort()[::-1][:top_n]
    top_scores = sims[top_idx]

    # 5. Trả về DataFrame kết quả
    result = data.iloc[top_idx][['id', 'Tiêu đề', 'Content']].copy()
    result["similarity"] = top_scores

    return result.reset_index(drop=True)


In [111]:
recommend_cosine_by_text("xe còn mới, máy êm, hao xăng ít, đời từ 2019 trở lên. Nếu có Vision hoặc Janus chạy dưới 10.000km thì càng tốt.", top_n=10)


,id,Tiêu đề,Content,similarity
0,6448,Xe janus xám đen bảng smartkey,Cần bán xe janus Do mua xe mới nên cần bán lại,0.405727
1,2207,Cần bán xe như hình một chủ,Xe còn tốt ít đi Một đời chủ Xem xe tại nhà Gi...,0.394077
2,6391,Gia đình cần đổi xe cần bán lại xe janus đời 2016,Gia đình cần đổi xe muốn bán lại xe janus đời ...,0.342975
3,6443,Cần bán xe tay ga Yamaha Janus,Xe chính chủ cần bán xe Yamaha Janus ai cần mu...,0.308978
4,3578,"Bán xe Vision 2018 - chạy 30.000km - Giá: 17,5...",Bán xe Vision 2018 nhà đang sử dụng - chạy 30....,0.291242
5,5031,"Cá nhân đổi xe, nên bán - Vespa 2014",Xe còn đi tốt Bán cá nhân Đã qua 1 đời chủ. Ti...,0.280459
6,5417,Bán,Xe chính chủ máy zin chưa rớt Odo ~10k km Xe n...,0.279231
7,2500,cần bán xe sh150i abs mới 99% trắng sx 2019,Mình cần bán xe sh150i 2019 abs màu trắng - xe...,0.261588
8,6393,janus,Janus đỏ đen đèn rất sáng giấy tờ đầy đủ,0.261481
9,3614,Bán Vision 215 màu đỏ biển tp thì chia lại cho e,Anh chị nào muốn lên đời hoặc nhà dư vision đờ...,0.257763
